## Get Project VM

In [ ]:
from chi import server, context
import chi, os, time, datetime, yaml
context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [ ]:
s = server.Server(
    f"node1-project16",
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium"
)

s.submit(idempotent=True)

### Install Docker

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

### Get the docker yaml files

In [ ]:
s.execute("git clone https://github.com/jasonmoon97/dynamic_nba_scheduling.git")

### Set up RClone

In [ ]:
s.execute("curl https://rclone.org/install.sh | sudo bash")
s.execute("sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf")

s.execute("mkdir -p ~/.config/rclone")

with open("clouds.yaml", "r") as file:
    secret = yaml.safe_load(file)

cloud_config = secret["clouds"]["chi_tacc"]
auth = cloud_config["auth"]

application_credential_id = auth["application_credential_id"]
application_credential_secret = auth["application_credential_secret"]

rclone_configs = f"""
[chi_tacc]
type = swift
user_id = 12c0ee0dd863e5fc52f1cb58899047dc431eba2bceb29f15984a05bf9c0bba8f
application_credential_id = {application_credential_id}
application_credential_secret = {application_credential_secret}
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
"""

safe_rclone = rclone_configs.strip()

s.execute(f""" cat <<EOF > ~/.config/rclone/rclone.conf
{safe_rclone}
EOF
""")

## ETL for Training

### Extract Data

In [ ]:
s.execute("docker compose -f ~/dynamic_nba_scheduling/data_engineering/docker/docker-etl.yaml run extract-data")

### Transform Data

In [ ]:
s.execute("docker compose -f ~/dynamic_nba_scheduling/data_engineering/docker/docker-etl.yaml run transform-data")

### Load Data to Object Store

In [ ]:
s.execute("export RCLONE_CONTAINER=object-persist-project16")
s.execute("docker compose -f ~/dynamic_nba_scheduling/data_engineering/docker/docker-etl.yaml run load-data")

## ETL for Online Eval Using Synthetic Data

### Use the extracted data in the docker volume, then generate syntehtic data

In [ ]:
s.execute("docker compose -f ~/dynamic_nba_scheduling/data_engineering/docker/docker-online.yaml run generate-data")

### Load the synthetic data

In [ ]:
s.execute("docker compose -f ~/dynamic_nba_scheduling/data_engineering/docker/docker-online.yaml run load-data")

## Mount the object Store

In [ ]:
s.execute("sudo mkdir -p /mnt/object")
s.execute("sudo chown -R cc /mnt/object")
s.execute("sudo chgrp -R cc /mnt/object")
s.execute("rclone mount chi_tacc:object-persist-project16 /mnt/object --read-only --allow-other --daemon")
s.execute("ls /mnt/object")

## Stop the docker volume and the VM

In [ ]:
s.execute("docker volume rm team16-etl_nbamodel")

In [ ]:
s.delete()